In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100)
import missingno as msno
import re
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
pd.set_option('max_columns', 120)
pd.set_option('max_colwidth', 5000)

In [51]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data_dictionary = pd.read_excel('../input/DataDictionary.xlsx',header=None, names = ["feature_name", "description"]) # Loading in the data dictionary

In [ ]:
train_df_dtypes = pd.DataFrame(train_df.dtypes,columns=['dtypes'])
train_df_dtypes = train_df_dtypes.reset_index()
train_df_dtypes['feature_name'] = train_df_dtypes['index']
train_df_dtypes = train_df_dtypes[['feature_name','dtypes']]

train_df_dtypes['first value'] = train_df.loc[0].values
preview = train_df_dtypes.merge(data_dictionary, on='feature_name',how='left')

In [ ]:
preview.head()

In [ ]:
preview[:19]

In [ ]:
preview[19:39]

In [ ]:
preview[39:]

In [52]:
def meta (dataframe):
    metadata = []
    for f in dataframe.columns:
    
        # Counting null values
        null = dataframe[f].isnull().sum()
    
        # Defining the data type 
        dtype = dataframe[f].dtype
    
        # Creating a Dict that contains all the metadata for the variable
        f_dict = {
            'varname': f,
            'nulls':null,
            'dtype': dtype
        }
        metadata.append(f_dict)

    meta = pd.DataFrame(metadata, columns=['varname','nulls', 'dtype'])
    meta.set_index('varname', inplace=True)
    meta=meta.sort_values(by=['nulls'],ascending=False)
    return meta

In [53]:
#Combine into data:
train_df['source']= 'train'
test_df['source'] = 'test'
dataset=pd.concat([train_df,test_df],ignore_index=True)
print(train_df.shape, test_df.shape, dataset.shape)

(709903, 65) (177476, 64) (887379, 65)


In [54]:
dataset['annual_inc'] = dataset['annual_inc'].fillna(dataset['annual_inc'].median())

In [55]:
# Variables: dti_joint, annual_inc_joint and verification_status_joint have so many null
# values as there are only 510 joint loans. Will replace NaN with 0 and 'None' for status.

dataset.dti_joint=dataset.dti_joint.replace(np.nan,0)
dataset.annual_inc_joint=dataset.annual_inc_joint.replace(np.nan,0)
dataset.verification_status_joint=dataset.verification_status_joint.replace(np.nan,'None')

In [56]:
variables1=['open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m','collections_12_mths_ex_med']

for e in variables1:
    dataset[e]=dataset[e].replace(np.nan,0)

Null values in these columns can't be replaced with 0 as it would mean that the last occurence was very recent. My understanding of these variables is that the key information is whether the specific action took place (delinquency, public record, worse rating), so I will turn these into binary categories of Yes (1), No (0).

In [57]:
dataset.loc[(dataset.mths_since_last_major_derog.notnull()),'derog']=1
dataset.loc[(dataset.mths_since_last_major_derog.isnull()),'derog']=0

dataset.loc[(dataset.mths_since_last_record.notnull()),'public_record']=1
dataset.loc[(dataset.mths_since_last_record.isnull()),'public_record']=0

dataset.drop(['mths_since_last_major_derog','mths_since_last_record'],axis=1,inplace=True)

Investigating variables connected to open_acc_6m which shows number of open trades in last 6 months. Variables open_il_6m, open_il_12m, open_il_24m, mths_since_rcnt_il, total_bal_il, il_util, open_rv_12m, open_rv_24m, max_bal_bc, all_util, inq_fi, total_cu_tl, inq_last_12m, collections_12_mths_ex_med have null values for the same rows - I will change them all to 0 as missing vaules show lack of open trades.

Investigating tot_coll_amt, tot_cur_bal, total_rev_hi_lim - these are three totals that have missing values for the same observations. I will change them to 0 as they should mean that the total is 0.

In [58]:
variables2=['tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']

for e in variables2:
    dataset[e]=dataset[e].replace(np.nan,0)

In [59]:
dataset.revol_util=dataset.revol_util.replace(np.nan,0)

In [60]:
dataset['earliest_cr_line'].mode()[0]

'Aug-2001'

There are seven variables with 3 missing values, this is such a small number that I will just replace NaN with most common values.

In [61]:
variables3=['total_acc','pub_rec','earliest_cr_line']

for e in variables3:
    dataset[e]=dataset[e].replace(np.nan,dataset[e].mode()[0])

In [62]:
pd.value_counts(dataset.loc[dataset['source']=='test'].pymnt_plan).to_frame()

,pymnt_plan
n,177473
y,3


In [63]:
pd.value_counts(dataset.verification_status).to_frame()

,verification_status
Source Verified,329558
Verified,291071
Not Verified,266750


In [16]:
pd.value_counts(dataset.verification_status_joint).to_frame()

,verification_status_joint
None,886868
Not Verified,283
Verified,167
Source Verified,61


In [17]:
pd.value_counts(dataset.application_type).to_frame()

,application_type
INDIVIDUAL,886868
JOINT,511


In [64]:
pd.value_counts(dataset.loan_status).to_frame()

,loan_status
Current,601779
Fully Paid,207723
Charged Off,45248
Late (31-120 days),11591
Issued,8460
In Grace Period,6253
Late (16-30 days),2357
Does not meet the credit policy. Status:Fully Paid,1988
Default,1219
Does not meet the credit policy. Status:Charged Off,761


In [22]:
meta(dataset)

,nulls,dtype
varname,,
desc,752565,object
acc_now_delinq,175205,float64
emp_title,50662,object
title,32,object
revol_bal,0,float64
term,0,object
sub_grade,0,object
source,0,object
revol_util,0,float64


In [68]:
print(dataset.loc[dataset['source']=='train'].shape, dataset.loc[dataset['source']=='test'].shape, dataset.shape)

(709903, 65) (177476, 65) (887379, 65)


In [69]:
# XGboost is good at dealing with numbers but definitely not good when dealing with string. 
# So the solution is neither we drop them or transform them.
drop_elements = ['pymnt_plan',
                 'issue_d',#The month which the loan was funded
                 'emp_title',
                 'desc',
                 'title',
                 'zip_code',
                 'addr_state'
                 ]
dataset.drop(drop_elements,axis=1,inplace=True)

#Handle datetime，将Feb-1985转换成1985-02-01
dataset['earliest_cr_line'] =  pd.to_datetime(dataset['earliest_cr_line'], format='%b-%Y')
# dataset['issue_d'] = pd.to_datetime(dataset['issue_d'], format='%b-%Y')

dataset['earliest_cr_line'] = dataset['earliest_cr_line'].map(lambda x: 12*x.year + x.month)
# dataset['issue_d'] = dataset['issue_d'].map(lambda x: 12*x.year + x.month)
dataset['nowMonth']=12*pd.to_datetime('today').year+pd.to_datetime('today').month
dataset['earliest_cr_line'] = dataset.nowMonth-dataset.earliest_cr_line
# dataset['issue_d'] = dataset.nowMonth-dataset.issue_d
dataset.drop(['nowMonth'],axis=1,inplace=True)

# Mapping term: 36 months->36
dataset['term']=dataset['term'].apply(lambda row:int(re.findall(r"\d+\.?\d*",row)[0]))

# Mapping Category feature
category_feature=['grade','sub_grade','home_ownership','verification_status','verification_status_joint','application_type','loan_status',
                  'purpose','initial_list_status','emp_length']
# 自然关系变量
Ordinal_Values = ['grade','sub_grade','emp_length']
# 离散变量->one_hot
Nominal_Values = ['loan_status', 'home_ownership','verification_status','verification_status_joint','purpose','initial_list_status']
dataset[category_feature]=dataset[category_feature].apply(LabelEncoder().fit_transform)

#One-Hot Coding¶
dataset = pd.get_dummies(dataset, columns=Nominal_Values)
    
#Handle datetime，将Feb-1985转换成1985-02-01
#     dataset['earliest_cr_line'] =  pd.to_datetime(dataset['earliest_cr_line'], format='%b-%Y')
#     dataset['issue_d'] = pd.to_datetime(dataset['issue_d'], format='%b-%Y')

In [70]:
dataset.columns

Index(['acc_now_delinq', 'all_util', 'annual_inc', 'annual_inc_joint',
       'application_type', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'dti', 'dti_joint', 'earliest_cr_line',
       'emp_length', 'funded_amnt', 'funded_amnt_inv', 'grade', 'il_util',
       'inq_fi', 'inq_last_12m', 'installment', 'int_rate', 'loan_amnt',
       'max_bal_bc', 'member_id', 'mths_since_rcnt_il', 'open_acc_6m',
       'open_il_12m', 'open_il_24m', 'open_il_6m', 'open_rv_12m',
       'open_rv_24m', 'out_prncp', 'out_prncp_inv', 'policy_code', 'pub_rec',
       'recoveries', 'revol_bal', 'revol_util', 'source', 'sub_grade', 'term',
       'tot_coll_amt', 'tot_cur_bal', 'total_acc', 'total_bal_il',
       'total_cu_tl', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int',
       'total_rec_late_fee', 'total_rec_prncp', 'total_rev_hi_lim', 'derog',
       'public_record', 'loan_status_0', 'loan_status_1', 'loan_status_2',
       'loan_status_3', 'loan_status_4', 'loan_status_5', 'loan_

In [71]:
print (train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709903 entries, 0 to 709902
Data columns (total 65 columns):
member_id                      709903 non-null int64
loan_amnt                      709903 non-null float64
funded_amnt                    709903 non-null float64
funded_amnt_inv                709903 non-null float64
term                           709903 non-null object
int_rate                       709903 non-null float64
installment                    709903 non-null float64
grade                          709903 non-null object
sub_grade                      709903 non-null object
emp_title                      668721 non-null object
emp_length                     709903 non-null object
home_ownership                 709903 non-null object
annual_inc                     709899 non-null float64
verification_status            709903 non-null object
issue_d                        709903 non-null object
loan_status                    709903 non-null object
pymnt_plan          

In [72]:
train = dataset.loc[dataset['source']=='train']
test = dataset.loc[dataset['source']=='test']

In [73]:
train.drop('source',axis=1,inplace=True)
test.drop(['source','acc_now_delinq'],axis=1,inplace=True)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [74]:
print(train.shape, test.shape)

(709903, 90) (177476, 89)


In [75]:
train.to_csv('../dump/train_modified.csv',index=False)
test.to_csv('../dump/test_modified.csv',index=False)